# Video Content Moderation  Example

This notebook contains snippets for interacting with the Amazon Rekognition Video's Content Modertion API.

Configure the global variables using the [VideoContentModeration.template](VideoContentModeration.template)'s stack **Outputs**.

1. Set the `video_input_bucket` to the generated Amazon S3 bucket
1. Set the `notification_channel_rolearn` to the generated AWS IAM role
1. Set the `notification_channel_snstopic_arn` to the generated Amazon SNS topic

In [1]:
import boto3

region_name = 'us-east-2'
video_input_bucket = 'videocontentmoderation-bucket83908e77-1emm2fx0ntdmb'
notification_channel_rolearn = 'arn:aws:iam::ACCOUNTID:role/VideoContentModeration-RekognitionRole7EFA3C18-3XT1J6HBCHTU'
notification_channel_snstopic_arn = 'arn:aws:sns:us-east-2:ACCOUNTID:VideoContentModeration-TopicBFC7AF6E-8sBlj2VzpAfm'
rekognition = boto3.client('rekognition', region_name=region_name)

Use the `StartContentModeration` API to begin asyncronously processing the [contentmoderationsample.mp4](contentmoderationsample.mp4) video clip.

In [2]:
moderation_job = rekognition.start_content_moderation(
    NotificationChannel={
        'RoleArn': notification_channel_rolearn,
        'SNSTopicArn': notification_channel_snstopic_arn
    },
    Video={
        'S3Object':{
            'Bucket':video_input_bucket,
            'Name':'contentmoderationsample.mp4'
        }
    }
)

moderation_job

{'JobId': '6f141c8a9d0dcb4154a79fedae85fa7bc37652581e18c0e8cb7925f1042d0050',
 'ResponseMetadata': {'RequestId': 'e5cf4d98-ff0c-4c5b-9184-1a356930e64d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e5cf4d98-ff0c-4c5b-9184-1a356930e64d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '76',
   'date': 'Sun, 08 Jan 2023 20:23:10 GMT'},
  'RetryAttempts': 0}}

You can check the moderation job's status using the `GetContentModeration` API.

In [3]:
response = rekognition.get_content_moderation(
    JobId=moderation_job['JobId']
)

The `get_complete_content_moderation_results` utility function will paginate the results and combine them into one logical response object.

In [6]:
def get_complete_content_moderation_results(jobId):
    next_token = None
    complete_response = None
    while True:
        if next_token:
            response = rekognition.get_content_moderation(            
                JobId=jobId,
                NextToken=next_token)        
            complete_response['ModerationLabels'].extend(response['ModerationLabels'])
        else:
            response = rekognition.get_content_moderation(
                JobId=jobId)
            complete_response = response
        
        if 'NextToken' in response:
            next_token = response['NextToken']
        else:
            break
    return complete_response

combined_response = get_complete_content_moderation_results(moderation_job['JobId'])
#print(combined_response)

Let's persist the `combined_response` into a JSON-formatted local file.

In [8]:
from json import dumps
with open('GetContentModerationVideoResponse.json', 'w') as f:
    f.write(dumps(combined_response,indent=2))

The response contains numerous labels. Let's aggregate them into a frequency table.

In [7]:
frequency={}
for label in combined_response['ModerationLabels']:
    name = label['ModerationLabel']['Name']
    if name in frequency:
        frequency[name] += 1
    else:
        frequency[name] =1

print(frequency)        

{'Female Swimwear Or Underwear': 35, 'Revealing Clothes': 35, 'Suggestive': 70, 'Alcohol': 24, 'Alcoholic Beverages': 21, 'Smoking': 2, 'Tobacco': 2, 'Gambling': 2, 'Barechested Male': 35, 'Drinking': 3}
